In [ ]:
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
import yaml
import sys
sys.path.insert(0, '../')
sys.path.insert(0, '../dataset')
import supershape as ss
import mesher
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300

In [ ]:
with open("datagen.yaml", "r") as file:
  config_data = yaml.safe_load(file)

# Shape parameters

In [ ]:
shape_yaml = config_data['SUPERSHAPE']
shape_extents = ss.SuperShapeExtents(
    a=ss.Extents(shape_yaml['min_a'], shape_yaml['max_a']),
    b=ss.Extents(shape_yaml['min_b'], shape_yaml['max_b']),
    m=ss.Extents(shape_yaml['min_m'], shape_yaml['max_m']),
    n1=ss.Extents(shape_yaml['min_n1'], shape_yaml['max_n1']),
    n2=ss.Extents(shape_yaml['min_n2'], shape_yaml['max_n2']),
    n3=ss.Extents(shape_yaml['min_n3'], shape_yaml['max_n3']),
    center_x=ss.Extents(shape_yaml['min_cx'], shape_yaml['max_cy']),
    center_y=ss.Extents(shape_yaml['min_cy'], shape_yaml['max_cy']),
)

# Generate shapes

In [ ]:
random_shape_param = ss.generate_random_super_shapes(
    config_data['DATASET']['num_samples'], shape_extents)

# Mesh to project polygons

In [ ]:
mesh_yaml = config_data['MESH']
polygons, pruned_shape_parameters = ss.super_shape_to_shapely_polygon(random_shape_param)
shape_density = ss.project_shapely_polygons_to_density(polygons,
                                                 mesh_yaml['nelx'],
                                                 mesh_yaml['nely'], True)

# Polygon area and perim

In [ ]:

shape_perim = ss.compute_shapely_polygon_perimeter(polygons)
shape_area = ss.compute_shapely_polygon_area(polygons)
normalized_shape_area = shape_area/(random_shape_param.domain_length_x*random_shape_param.domain_length_y)    #Normalize to fit within unit cell of 1*1
normalized_shape_perim = shape_perim/(random_shape_param.domain_length_x + random_shape_param.domain_length_y)

# Plot some of the polygons

In [ ]:
n_imgs_x, n_imgs_y = 4, 4
rand_img_idxs = np.random.randint(0, shape_density.shape[0], (n_imgs_x*n_imgs_y,))
rand_mstr_imgs = shape_density[rand_img_idxs,:]
rand_shape_area = normalized_shape_area[rand_img_idxs]
rand_shape_perim = normalized_shape_perim[rand_img_idxs]

_, axs = plt.subplots(n_imgs_x, n_imgs_y)
axs = axs.flatten()
for i, ax in enumerate(axs):
    im = ax.imshow(rand_mstr_imgs[i,:].T,
              cmap='coolwarm')
    ax.set_title(f'{rand_shape_perim[i]:.2F}')
    ax.axis('off')
cbar = plt.colorbar(im, ax=axs)
cbar.set_label('Colorbar Label')
plt.show()

# Save Dataset for Homogenization

In [ ]:
shape_params = pruned_shape_parameters.to_stacked_array()

In [ ]:
dataset_num  = config_data['DATASET']['dataset_num']
scipy.io.savemat(f'../dataset/mstr_shape_parameters_{dataset_num}.mat', 
                 {'mstr_shape_parameters': shape_params})
scipy.io.savemat(f'../dataset/mstr_images_{dataset_num}.mat', 
                 {'mstr_images': shape_density})
scipy.io.savemat(f'../dataset/mstr_area_{dataset_num}.mat', 
                 {'mstr_area': normalized_shape_area})
scipy.io.savemat(f'../dataset/mstr_perim_{dataset_num}.mat', 
                 {'mstr_perim': normalized_shape_perim})